In [3]:
import json, random, re, collections, itertools, base64, sys
from copy import deepcopy
from pathlib import Path
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from operator import itemgetter as at
from IPython.core.display import display, HTML
from ipywidgets import interact
display_html = lambda x: display(HTML(str(x)))
from pprint import pprint as pp
ls = lambda path: list(map(str, path.iterdir()))

sys.path.append('../src')
mturk_path = Path("../mturk/results")
data_path = Path("../data")
preprocessed_path = Path("../preprocessed")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
ls(mturk_path)

['../mturk/results/Batch_4258572_batch_results.csv:Zone.Identifier',
 '../mturk/results/Batch_4258572_batch_results.csv']

In [35]:
def load_results(batch):
    df = pd.read_csv(str(next(mturk_path.rglob(f"*{batch}*.csv"))))
    df = df[[c for c in df.columns if "Answer." in c or 'Id' in c]]
    df = df.rename(columns={c:c.split('.',1)[1] for c in df.columns if '.' in c})
    return df.set_index("AssignmentId")
batch = 4258572
df=load_results(batch)
df

,HITId,HITTypeId,WorkerId,events,feedback,id,status
AssignmentId,,,,,,,
320DUZ38GDHZZVYQ4B55MLMLILKGJ5,3BVS8WK9R5Q5QJ9FG5NQKPK1W21IBO,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,AORHXBTOCXFUK,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",thermometer,223296,-1
3GNCZX450OIUBFOLDOIFNVXUTKAAPA,3BVS8WK9R5Q5QJ9FG5NQKPK1W21IBO,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,A3BMEVFSETVOUR,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},223296,1
3OS46CRSLLU6TX9E9NA2DIKQJYD6VB,3J94SKDELNKMTSCKNHV2Q64WU17D5G,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,AX8EX2QI5HIQQ,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",good,11864,-3
3HRMW88U1CLSPZTQT6TUHLRMFV1M0H,3D1UCPY6HL47YITPSQWTPHSB5EJ83F,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,A2CKYCHQGXUI10,"[[],[],[]]",good,117821,-10
3RYC5T2D79OMAWA2FZ41ILM4TXXRPG,3UQ1LLR27F32XACTVUK7HUSNVOAALZ,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,A1GGOZPHYU0OC0,"[[],[],[],[]]",{},45580,1
3TPWUS5F8FW8O3IK9OS1LKVEMMDCWS,3BS6ERDLA82ICWCYS057ICIWQWID6W,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,A2CKYCHQGXUI10,"[[],[],[],[]]",{},13992,-4
3DBQWDE4YCTX2O0R2DE6BVCD1YJN5A,3XUSYT70JYWNYLQJQBPQ9B85DTW0DI,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,ABMPX2Y2IRBMG,"[[],[],[],[],[]]",{},86245,1
338JKRMM2CU2YYQXUBAAEMV37VNAHW,3EN4YVUOVHN8YXWM3J08LNQTSKDXJN,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,A2SBN9B6Q3DN2X,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},100386,1
3EKVH9QME4ZWHFQRV0Y5HJX73MGD2B,3EN4YVUOVHN8YXWM3J08LNQTSKDXJN,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,A1AYL5V5GI9HC1,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},100386,1


In [48]:
assignment="3HRMW88U1CLSPZTQT6TUHLRMFV1M0H"
idx = df.loc[assignment, "id"]
status = df.loc[assignment, "status"]
display_html(f'<a href="http://127.0.0.1:8080/annotate/{batch}/{assignment}" target="_new">{idx} - {status}</a>')

In [33]:
str(next(mturk_path.rglob("*4258572*.csv")))

'../mturk/results/Batch_4258572_batch_results.csv'